In [ ]:
# Importar bibliotecas necesarias
import cv2  # OpenCV para el procesamiento de imágenes
import torch  # PyTorch para el modelo YOLOv5
import pygame  # Pygame para la reproducción de sonido
import time  # Biblioteca de tiempo para el seguimiento del tiempo
from pushbullet import Pushbullet

In [ ]:
# Cargar el modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)

In [ ]:
# Inicializar pygame para reproducción de sonido
pygame.init()
alarma = pygame.mixer.Sound(r"D:\LaSalle\3erSemestre\MashineLearning\3erParcial\NuevoFinalProyecto\Alarma.wav")  # Reemplazar con la ruta de tu archivo de sonido

# Variable para rastrear la última vez que se detectó un intruso
ultimo_intruso = None
tiempo_alarma = 2  # Duración para mantener la alarma después de que el intruso se haya ido, en segundos

In [ ]:
# Función para verificar si el objeto detectado está en la mitad izquierda del fotograma
def pantalla(det, frame_width):
    xmin = int(det[0])
    xmax = int(det[2])

    # Comprueba si el centro del objeto detectado está en la mitad izquierda del fotograma
    center_x = (xmin + xmax) / 2
    return center_x

In [ ]:
# Alarma al celular
pb = Pushbullet('o.gt8Amm4ifh3hqUlbX8aNZQRWEy7jkC9u')

def send_notification():
    pb.push_note("Alerta de Seguridad", "Intruso detectado!")

In [ ]:
# Función para procesar cada fotograma
def process_frame(frame):
    global ultimo_intruso

   # Convertir el formato de color de BGR a RGB (formato usado por YOLOv5)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Realizar la detección de objetos usando YOLOv5
    results = model(frame_rgb)

    # Obtener el ancho del fotograma
    frame_width = frame.shape[1]
    
    # Variable para rastrear si se detecta un intruso en el fotograma actual
    intruso = False

    # Obtener resultados de la detección de personas
    results = results.xyxy[0].numpy()
    person_detections = [det for det in results if int(det[5]) == 1]

    # Iterar sobre las detecciones de personas y dibujar rectángulos y etiquetas en el fotograma
    for det in person_detections:
        if pantalla(det, frame_width):
            xmin, ymin, xmax, ymax = map(int, det[:4])
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
            cv2.putText(frame, 'Intruso', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            intruso = True

    # Actualizar el tiempo del último intruso detectado
    if intruso:
        if ultimo_intruso is None or (time.time() - ultimo_intruso) >= tiempo_alarma:
            send_notification()  # Enviar notificación al detectar un intruso
        ultimo_intruso = time.time()

    # Reproducir o detener la alarma según la detección y el tiempo
    current_time = time.time()
    if ultimo_intruso and (current_time - ultimo_intruso < tiempo_alarma):
        if not pygame.mixer.get_busy():  # Comprobar si el sonido ya está reproduciéndose
            pygame.mixer.Sound.play(alarma)
    else:
        pygame.mixer.stop()  # Detener la alarma si no hay intrusos o el tiempo ha pasado

    return frame

In [ ]:
# Inicializar la cámara (usar 0 para la cámara predeterminada)
camera = cv2.VideoCapture(0)

# Bucle principal para capturar y procesar los fotogramas
while True:
    ret, frame = camera.read()
    if not ret:
        break

    # Procesar el fotograma
    frame = process_frame(frame)

    # Mostrar el resultado en una ventana
    cv2.imshow('Deteccion de Intrusos', frame)

    # Presionar 'q' para salir del bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# Liberar la cámara y cerrar la ventana
camera.release()
cv2.destroyAllWindows()